In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 152.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 179.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 198.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 40.8

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import snapshot_download
import os


local_dir = snapshot_download(
    repo_id="huytran98/fpt-open-audio-noised",
    repo_type="dataset",
    local_dir="./fpt-dataset"
)

# Kiểm tra cấu trúc thư mục
print("Dataset directory:", local_dir)
print("\nFiles and directories:")
!ls -R {local_dir}

# Đọc transcript
transcript_path = os.path.join(local_dir, "transcript.txt")
with open(transcript_path, 'r', encoding='utf-8') as f:
    transcript = f.read()
print("\nFirst few lines of transcript:")
print(transcript.split('\n')[:5])

# Đường dẫn đến thư mục audio
noisy_audio_dir = os.path.join(local_dir, "noisy-data-500")
clean_audio_dir = os.path.join(local_dir, "clean-data-500")

# Liệt kê một vài file audio
print("\nNoisy audio files:")
print(os.listdir(noisy_audio_dir)[:5])

Fetching 603 files:   0%|          | 0/603 [00:00<?, ?it/s]

Dataset directory: /content/fpt-dataset

Files and directories:
/content/fpt-dataset:
clean-data-500	noisy-data-500	README.md  transcript.txt

/content/fpt-dataset/clean-data-500:
FPTOpenSpeechData_Set001_V0.1_000201.mp3  FPTOpenSpeechData_Set001_V0.1_000351.mp3
FPTOpenSpeechData_Set001_V0.1_000202.mp3  FPTOpenSpeechData_Set001_V0.1_000352.mp3
FPTOpenSpeechData_Set001_V0.1_000203.mp3  FPTOpenSpeechData_Set001_V0.1_000353.mp3
FPTOpenSpeechData_Set001_V0.1_000204.mp3  FPTOpenSpeechData_Set001_V0.1_000354.mp3
FPTOpenSpeechData_Set001_V0.1_000205.mp3  FPTOpenSpeechData_Set001_V0.1_000355.mp3
FPTOpenSpeechData_Set001_V0.1_000206.mp3  FPTOpenSpeechData_Set001_V0.1_000356.mp3
FPTOpenSpeechData_Set001_V0.1_000207.mp3  FPTOpenSpeechData_Set001_V0.1_000357.mp3
FPTOpenSpeechData_Set001_V0.1_000208.mp3  FPTOpenSpeechData_Set001_V0.1_000358.mp3
FPTOpenSpeechData_Set001_V0.1_000209.mp3  FPTOpenSpeechData_Set001_V0.1_000359.mp3
FPTOpenSpeechData_Set001_V0.1_000210.mp3  FPTOpenSpeechData_Set001_V0.1_0

In [ ]:
from datasets import Dataset, Audio
import pandas as pd
import os

def create_dataset_from_files(transcript_path, audio_dir, use_noisy=True):
    """
    Create dataset from local files
    """

    df = pd.read_csv(transcript_path, sep='|', names=['audio_file', 'text', 'duration'])

    def get_audio_path(row):
        base_name = os.path.basename(row['audio_file'])
        if use_noisy:
            audio_name = base_name.replace('.mp3', '_file_noise.mp3')
        else:
            audio_name = base_name
        return os.path.join(audio_dir, audio_name)

    df['audio'] = df.apply(get_audio_path, axis=1)

    df['exists'] = df['audio'].apply(os.path.exists)
    valid_df = df[df['exists']].reset_index(drop=True)

    print(f"Total files in transcript: {len(df)}")
    print(f"Valid files found: {len(valid_df)}")

    dataset = Dataset.from_pandas(valid_df)

    # From 48 kHz to 16 kHz
    dataset = dataset.cast_column('audio', Audio(sampling_rate=16000))

    return dataset


In [ ]:
try:
    base_dir = "./fpt-dataset"
    transcript_path = os.path.join(base_dir, "transcript.txt")
    audio_dir = os.path.join(base_dir, "clean-data-500")

    # Create dataset from local files
    dataset = create_dataset_from_files(
        transcript_path=transcript_path,
        audio_dir=audio_dir,
        use_noisy=False # clean data
    )

    # Train/test
    splits = dataset.train_test_split(test_size=0.2, seed=42)

    print("\nDataset splits:")
    print(f"Train: {len(splits['train'])} examples")
    print(f"Test: {len(splits['test'])} examples")

    sample = splits['train'][0]
    print("\nSample:")
    print(f"Audio path: {sample['audio']}")
    print(f"Text: {sample['text']}")
    print(f"Duration: {sample['duration']}")

except Exception as e:
    print(f"Error: {str(e)}")

Total files in transcript: 25921
Valid files found: 300

Dataset splits:
Train: 240 examples
Test: 60 examples

Sample:
Audio path: {'path': './fpt-dataset/clean-data-500/FPTOpenSpeechData_Set001_V0.1_000241.mp3', 'array': array([ 3.13982950e-04,  1.90811101e-04,  7.50760228e-05, ...,
        3.21989763e-04, -5.89779171e-04, -2.63103371e-04]), 'sampling_rate': 16000}
Text: đều tắc do ngập nước
Duration: 0.00000-1.72898


In [ ]:
print("Columns before:", splits['train'].column_names)
splits = splits.remove_columns(['duration', 'audio_file', 'exists'])
splits = splits.rename_column('text', 'sentence')
print("Columns after:", splits['train'].column_names)

Columns before: ['audio_file', 'text', 'duration', 'audio', 'exists']
Columns after: ['sentence', 'audio']


In [ ]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Vietnamese", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
# Test tokenizer
input_str = splits["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 đều tắc do ngập nước
Decoded w/ special:    <|startoftranscript|><|vi|><|transcribe|><|notimestamps|>đều tắc do ngập nước<|endoftext|>
Decoded w/out special: đều tắc do ngập nước
Are equal:             True


In [ ]:
# Test tokenizer
input_str = splits["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 đều tắc do ngập nước
Decoded w/ special:    <|startoftranscript|><|vi|><|transcribe|><|notimestamps|>đều tắc do ngập nước<|endoftext|>
Decoded w/out special: đều tắc do ngập nước
Are equal:             True


In [ ]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Vietnamese", task="transcribe")


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
splits = splits.map(prepare_dataset, remove_columns=splits.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/240 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
model_id="openai/whisper-tiny"
# model_id="huytran98/whisper-tiny-clean"

In [ ]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained(model_id)


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "vietnamese"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="huytran98/whisper-tiny-clean",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=splits["train"],
    eval_dataset=splits["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-26-8bb1332dfb1a>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
